Itay Koren Project

In [4]:
pip install psycopg2 -U

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import reflection
import psycopg2

### Access to the DB <a class="anchor" id="chapter2"></a>

In [2]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-sales-data-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})
inspector = reflection.Inspector.from_engine(engine)


<ipython-input-2-60b087478d5e>:14: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = reflection.Inspector.from_engine(engine)


In [3]:
inspector.get_columns('rep_sales')

[{'name': 'OrderDate',
  'type': TIMESTAMP(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Region',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Rep',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Item',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Units',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Unit_Cost',
  'type': NUMERIC(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'Total',
  'type': NUMERIC(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

#  Table Queries <a class="anchor" id="chapter3"></a>

function that takes a query and return dataframe for general use

In [4]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)
    

rep_sales table:

        OrderDate: date when the order was placed

        Region: geographical area in which the sale was made.

        Rep: sales representative's name

        Item: name of the item sold

        Units: number of units sold

        UnitCost: cost of one unit

        Total: total cost of the order - Units x UnitCost

Item table:

        Item: name of the item

        MinPrice: minimum price that the item can be sold.

1. What is the total income in the data?

**You mean rep_sales? since that is the only table that has income"**

In [10]:
query = (
'SELECT SUM("Total") '
'FROM rep_sales '
)

queryResult(query)

,sum
0,28867.97



2. Which sales rep brought the most income?


In [54]:
query = '''
SELECT
    "Rep",
    sum
FROM
 (SELECT
        "Rep",
        SUM("Total")
    FROM
        rep_sales
    GROUP BY
        "Rep") as subq 
ORDER BY
    sum DESC
LIMIT 1;
'''

queryResult(query)

,Rep,sum
0,Kivell,3554.23



3. Which item brought the most income?

In [56]:
query = '''
SELECT
    "Item",
    sum
FROM
 (SELECT
        "Item",
        SUM("Total")
    FROM
        rep_sales
    GROUP BY
        "Item") as subq 
ORDER BY
    sum DESC
LIMIT 1;
'''

queryResult(query)

,Item,sum
0,Binder,8229.12




4. Which region sold the most desks?


In [66]:
# improvements: 
# Remove Item and give SUM("Units") as desk
# now I can remove group by Item
query = '''
SELECT
    "Region",
    "Item",
    SUM("Units")
FROM
    rep_sales

WHERE
    "Item" = 'Desk'
GROUP BY
    "Region", "Item"
ORDER BY
    sum DESC
'''

queryResult(query)

,Region,Item,sum
0,West,Desk,9
1,Central,Desk,7
2,East,Desk,7



5. Which sales rep sold the highest average price a piece for the item pen?


In [75]:
query = '''
SELECT
    "Rep",
    AVG("Unit_Cost") as avg_unit_cost
FROM
    rep_sales
WHERE
    "Item" = 'Pen'
GROUP BY
    "Rep"
ORDER BY
    avg_unit_cost DESC
'''

queryResult(query)

,Rep,avg_unit_cost
0,Parent,19.99
1,Gill,10.74
2,Jones,8.99
3,Howard,4.99
4,Joe,2.19
5,Sorvino,1.99
6,Jardine,1.79
7,Thompson,1.59
8,Andrews,1.50



6. Which sales rep sold the most units between April – September of 2020?

**Including September?**


In [78]:
query = '''
SELECT
    "Rep",
    SUM("Units") as sum_units
FROM
    rep_sales
WHERE
    "OrderDate"::date BETWEEN '2020-04-01' AND '2020-09-30'
GROUP BY
    "Rep"
ORDER BY
    sum_units DESC
'''

queryResult(query)

,Rep,sum_units
0,Andrews,309
1,Gill,256
2,Thompson,245
3,Howard,99
4,Kivell,94
5,Jones,86
6,Sorvino,79
7,Morgan,55
8,Joe,38



7. Which sales rep had the highest price difference from the minimum price on the item pen set?


In [133]:
query = '''
SELECT
    subq."Rep",
    subq.max_price - item."MinPrice" as price_diff
FROM
    item
    INNER JOIN (SELECT
        "Rep",
        "Item",
        Max("Unit_Cost") as max_price
     FROM
        rep_sales
    
     GROUP BY
        "Rep", "Item"
     ) as subq ON subq."Item" = item."Item"
WHERE
    item."Item" = 'Pen Set'
ORDER BY
    price_diff DESC
'''

queryResult(query)

,Rep,price_diff
0,Kivell,12.01
1,Kivell,12.01
2,Jones,11.00
3,Jones,11.00
4,Parent,8.00
5,Parent,8.00
6,Morgan,7.50
7,Morgan,7.50
8,Thompson,4.00
9,Thompson,4.00



8. Was there a sales rep that sold an item below the minimum price? If so, which item was it and what was the percent difference from the minimum price?


In [95]:
# Improvments:

# from rep_sales inner join (select distinct * from item) as x on rep_sales."Item"=x."Item"
# Using distinct is like doing group by but will save the name of the columns
query = '''
SELECT
    subq."Rep",
    subq."Item",
    100 - (subq.min_price / item."MinPrice")*100 as precent_diff_from_min_price
FROM
    item
    INNER JOIN (SELECT
        "Rep",
        "Item",
        MIN("Unit_Cost") as min_price
     FROM
        rep_sales
    
     GROUP BY
        "Rep", "Item"
     ) as subq ON subq."Item" = item."Item"
WHERE 
    subq."min_price" < item."MinPrice" 
ORDER BY
    precent_diff_from_min_price DESC
'''

queryResult(query)

,Rep,Item,precent_diff_from_min_price
0,Gill,Pen,14.857143
1,Gill,Pen,14.857143
2,Andrews,Pen,14.285714
3,Andrews,Pen,14.285714
4,Thompson,Pen,9.142857
5,Thompson,Pen,9.142857



9. Which region had the highest share of  price difference ?

(where percent price different is the amount above min price divided by the total amount from that region)


,region,item,item_by_region_total,region_total,unit_total_in_region_devided_region_total
0,Central,Pen,6210.76,15167.92,0.409467
1,West,Binder,9719.30,4982.88,1.950539
2,East,Binder,25650.50,8717.17,2.942526
3,East,Marker Set,10255.00,8717.17,1.176414
4,West,Pen,1272.76,4982.88,0.255427
5,East,Pencil,2310.62,8717.17,0.265065
6,Central,Binder,44519.26,15167.92,2.935093
7,Central,Pencil,7589.40,15167.92,0.500359
8,West,Pen Set,3146.50,4982.88,0.631462
9,East,Pen,12073.10,8717.17,1.384979



10. For each sale above the minimum price the rep gets 10% commission of the total price as a bonus. Which 3 sales reps have earned the most commission (show their commission as well)?